In [3]:
import arcpy ##Arcgis License

ModuleNotFoundError: No module named 'arcpy'

In [ ]:
arcpy.env.workspace = 'C:/data'

In [ ]:
#Create LAS Dataset (3D Analyst or Spatial Analyst)
arcpy.CreateLasDataset_management(input[folder], out_las_dataset, 'RECURSION')

In [1]:
#Classify LAS Building (3D Analyst)
arcpy.ClassifyLasBuilding_3d(out_las_dataset, minHeight='6.56 Feet', minArea='47.6 SquareFeet', compute_stats=True)

NameError: name 'arcpy' is not defined

In [ ]:
#LAS Dataset to Raster (Spatial Analyst or 3D Analyst) 0.3048: covert feet to meter
arcpy.LasDatasetToRaster_conversion(out_las_dataset, raster, 'ELEVATION', "BINNING AVERAGE NATURAL_NEIGHBOR",
                                   "FLOAT", "CELLSIZE", ##10##, 0.3048)